In [46]:
import pandas as pd
import json
import os
import numpy as np
import re

from multiprocessing import Pool, cpu_count
import time

In [47]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
#from google.colab import auth
#auth.authenticate_user()
#from googleapiclient.discovery import build
#drive_service = build('drive', 'v3')

In [48]:
S_DIR = r'/content/gdrive/My Drive/data_chan/'

with open(os.path.join(S_DIR, 'farmers-protest-tweets-2021-2-4.json')) as json_file:
    data = json_file.readlines()
    data = list(map(json.loads, data))


In [49]:
data[0]

{'url': 'https://twitter.com/ArjunSinghPanam/status/1364506249291784198',
 'date': '2021-02-24T09:23:35+00:00',
 'content': 'The world progresses while the Indian police and Govt are still trying to take India back to the horrific past through its tyranny. \n\n@narendramodi @DelhiPolice Shame on you. \n\n#ModiDontSellFarmers \n#FarmersProtest \n#FreeNodeepKaur https://t.co/es3kn0IQAF',
 'renderedContent': 'The world progresses while the Indian police and Govt are still trying to take India back to the horrific past through its tyranny. \n\n@narendramodi @DelhiPolice Shame on you. \n\n#ModiDontSellFarmers \n#FarmersProtest \n#FreeNodeepKaur twitter.com/ravisinghka/st…',
 'id': 1364506249291784198,
 'user': {'username': 'ArjunSinghPanam',
  'displayname': 'Arjun Singh Panam',
  'id': 45091142,
  'description': 'Global Citizen, Actor, Director: Sky is the roof above my head, the world is the road I travel, love is my food & mother earth is my bed. Roy in @CosmosMovie',
  'rawDescription':

In [50]:
df = pd.DataFrame(data)

In [ ]:
df.columns

Index(['url', 'date', 'content', 'renderedContent', 'id', 'user', 'outlinks',
       'tcooutlinks', 'replyCount', 'retweetCount', 'likeCount', 'quoteCount',
       'conversationId', 'lang', 'source', 'sourceUrl', 'sourceLabel', 'media',
       'retweetedTweet', 'quotedTweet', 'mentionedUsers'],
      dtype='object')

In [51]:
df.dtypes

url                object
date               object
content            object
renderedContent    object
id                  int64
user               object
outlinks           object
tcooutlinks        object
replyCount          int64
retweetCount        int64
likeCount           int64
quoteCount          int64
conversationId      int64
lang               object
source             object
sourceUrl          object
sourceLabel        object
media              object
retweetedTweet     object
quotedTweet        object
mentionedUsers     object
dtype: object

In [52]:
df.head()

,url,date,content,renderedContent,id,user,outlinks,tcooutlinks,replyCount,retweetCount,...,quoteCount,conversationId,lang,source,sourceUrl,sourceLabel,media,retweetedTweet,quotedTweet,mentionedUsers
0,https://twitter.com/ArjunSinghPanam/status/136...,2021-02-24T09:23:35+00:00,The world progresses while the Indian police a...,The world progresses while the Indian police a...,1364506249291784198,"{'username': 'ArjunSinghPanam', 'displayname':...",[https://twitter.com/ravisinghka/status/136415...,[https://t.co/es3kn0IQAF],0,0,...,0,1364506249291784198,en,"<a href=""http://twitter.com/download/iphone"" r...",http://twitter.com/download/iphone,Twitter for iPhone,None,None,{'url': 'https://twitter.com/RaviSinghKA/statu...,"[{'username': 'narendramodi', 'displayname': '..."
1,https://twitter.com/PrdeepNain/status/13645062...,2021-02-24T09:23:32+00:00,#FarmersProtest \n#ModiIgnoringFarmersDeaths \...,#FarmersProtest \n#ModiIgnoringFarmersDeaths \...,1364506237451313155,"{'username': 'PrdeepNain', 'displayname': 'Pra...",[],[],0,0,...,0,1364506237451313155,en,"<a href=""http://twitter.com/download/android"" ...",http://twitter.com/download/android,Twitter for Android,[{'thumbnailUrl': 'https://pbs.twimg.com/ext_t...,None,None,"[{'username': 'Kisanektamorcha', 'displayname'..."
2,https://twitter.com/parmarmaninder/status/1364...,2021-02-24T09:23:22+00:00,ਪੈਟਰੋਲ ਦੀਆਂ ਕੀਮਤਾਂ ਨੂੰ ਮੱਦੇਨਜ਼ਰ ਰੱਖਦੇ ਹੋਏ \nਮੇ...,ਪੈਟਰੋਲ ਦੀਆਂ ਕੀਮਤਾਂ ਨੂੰ ਮੱਦੇਨਜ਼ਰ ਰੱਖਦੇ ਹੋਏ \nਮੇ...,1364506195453767680,"{'username': 'parmarmaninder', 'displayname': ...",[],[],0,0,...,0,1364506195453767680,pa,"<a href=""http://twitter.com/download/android"" ...",http://twitter.com/download/android,Twitter for Android,None,None,None,None
3,https://twitter.com/anmoldhaliwal/status/13645...,2021-02-24T09:23:16+00:00,@ReallySwara @rohini_sgh watch full video here...,@ReallySwara @rohini_sgh watch full video here...,1364506167226032128,"{'username': 'anmoldhaliwal', 'displayname': '...",[https://youtu.be/-bUKumwq-J8],[https://t.co/wBPNdJdB0n],0,0,...,0,1364350947099484160,en,"<a href=""https://mobile.twitter.com"" rel=""nofo...",https://mobile.twitter.com,Twitter Web App,[{'thumbnailUrl': 'https://pbs.twimg.com/ext_t...,None,None,"[{'username': 'ReallySwara', 'displayname': 'S..."
4,https://twitter.com/KotiaPreet/status/13645061...,2021-02-24T09:23:10+00:00,#KisanEktaMorcha #FarmersProtest #NoFarmersNoF...,#KisanEktaMorcha #FarmersProtest #NoFarmersNoF...,1364506144002088963,"{'username': 'KotiaPreet', 'displayname': 'Pre...",[],[],0,0,...,0,1364506144002088963,und,"<a href=""http://twitter.com/download/iphone"" r...",http://twitter.com/download/iphone,Twitter for iPhone,[{'previewUrl': 'https://pbs.twimg.com/media/E...,None,None,None


Se identifican tipos de datos que posiblemente se pueden reasignar para reducir consumo de memoria

#Punto 1

In [ ]:
df[['date','user']].head()

,date,user
0,2021-02-24T09:23:35+00:00,"{'username': 'ArjunSinghPanam', 'displayname':..."
1,2021-02-24T09:23:32+00:00,"{'username': 'PrdeepNain', 'displayname': 'Pra..."
2,2021-02-24T09:23:22+00:00,"{'username': 'parmarmaninder', 'displayname': ..."
3,2021-02-24T09:23:16+00:00,"{'username': 'anmoldhaliwal', 'displayname': '..."
4,2021-02-24T09:23:10+00:00,"{'username': 'KotiaPreet', 'displayname': 'Pre..."


In [ ]:
df = df[['date','user']]

In [ ]:
df[['date','user']].dtypes

date    object
user    object
dtype: object

In [ ]:
df['date'] = pd.to_datetime(df['date']).dt.date

In [ ]:
def find_user(user_object):
    return user_object.get('username')

In [ ]:
df['user'] = df.apply(lambda row: row.user.get('username'), axis=1)

In [ ]:
df.head()

,date,user
0,2021-02-24,ArjunSinghPanam
1,2021-02-24,PrdeepNain
2,2021-02-24,parmarmaninder
3,2021-02-24,anmoldhaliwal
4,2021-02-24,KotiaPreet


In [ ]:
 dates = df.groupby(['date']).agg(count=("date", 'count')).reset_index().sort_values(by=['count'], ascending=False).head(10)['date'].to_list()
 df = df[df['date'].isin(dates)]

In [ ]:
df2['date'].value_counts()

2021-02-12    12347
2021-02-13    11296
2021-02-17    11087
2021-02-16    10443
2021-02-14    10249
2021-02-18     9625
2021-02-15     9197
2021-02-20     8502
2021-02-23     8417
2021-02-19     8204
Name: date, dtype: int64

In [ ]:
 df.groupby(['date','user']).agg(count=("user", 'count')).sort_values(by=['count'], ascending=False)

count
date       user                  
2021-02-19 Preetm91           267
2021-02-18 neetuanjle_nitu    195
2021-02-17 RaaJVinderkaur     185
2021-02-13 MaanDee08215437    178
2021-02-12 RanbirS00614606    176
...                           ...
2021-02-15 PulseHuman           1
           Profdilipmandal      1
           ProfSGB1             1
           Priyanka744074       1
2021-02-23 zlz_raa              1

[44159 rows x 1 columns]

In [ ]:
df_2 = pd.crosstab(df['date'], df['user'])

In [ ]:
list(df_2.idxmax(axis=1).to_dict().items())

[(datetime.date(2021, 2, 12), 'RanbirS00614606'),
 (datetime.date(2021, 2, 13), 'MaanDee08215437'),
 (datetime.date(2021, 2, 14), 'rebelpacifist'),
 (datetime.date(2021, 2, 15), 'jot__b'),
 (datetime.date(2021, 2, 16), 'jot__b'),
 (datetime.date(2021, 2, 17), 'RaaJVinderkaur'),
 (datetime.date(2021, 2, 18), 'neetuanjle_nitu'),
 (datetime.date(2021, 2, 19), 'Preetm91'),
 (datetime.date(2021, 2, 20), 'MangalJ23056160'),
 (datetime.date(2021, 2, 23), 'Surrypuria')]

In [ ]:
def punto1():
  S_DIR = r'/content/gdrive/My Drive/data_chan/'

  with open(os.path.join(S_DIR, 'farmers-protest-tweets-2021-2-4.json')) as json_file:
      data = json_file.readlines()
      data = list(map(json.loads, data))
  df = pd.DataFrame(data)
  df = df[['date','user']]
  df['date'] = pd.to_datetime(df['date']).dt.date
  df['user'] = df.apply(lambda row: row.user.get('username'), axis=1)
  dates = df.groupby(['date']).agg(count=("date", 'count')).reset_index().sort_values(by=['count'], ascending=False).head(10)['date'].to_list()
  df = df[df['date'].isin(dates)]
  df = pd.crosstab(df['date'], df['user'])
  return list(df.idxmax(axis=1).to_dict().items())


In [ ]:
punto1()

[(datetime.date(2021, 2, 12), 'RanbirS00614606'),
 (datetime.date(2021, 2, 13), 'MaanDee08215437'),
 (datetime.date(2021, 2, 14), 'rebelpacifist'),
 (datetime.date(2021, 2, 15), 'jot__b'),
 (datetime.date(2021, 2, 16), 'jot__b'),
 (datetime.date(2021, 2, 17), 'RaaJVinderkaur'),
 (datetime.date(2021, 2, 18), 'neetuanjle_nitu'),
 (datetime.date(2021, 2, 19), 'Preetm91'),
 (datetime.date(2021, 2, 20), 'MangalJ23056160'),
 (datetime.date(2021, 2, 21), 'Surrypuria'),
 (datetime.date(2021, 2, 22), 'preetysaini321'),
 (datetime.date(2021, 2, 23), 'Surrypuria'),
 (datetime.date(2021, 2, 24), 'preetysaini321')]

### Memory

In [ ]:
def load_data_1():
  S_DIR = r'/content/gdrive/My Drive/data_chan/'

  with open(os.path.join(S_DIR, 'farmers-protest-tweets-2021-2-4.json')) as json_file:
      data = json_file.readlines()
      data = list(map(json.loads, data))
  return data
# 1500 memoria
#27 segundos

def load_data():
  S_DIR = r'/content/gdrive/My Drive/data_chan/'

  with open(os.path.join(S_DIR, 'farmers-protest-tweets-2021-2-4.json')) as json_file:
      data = json_file.readlines()
      data = list(map(json.loads, data))
      data = [{k:d[k] if k != 'user' else d[k].get('username') for k in ["date","user"]} for d in data]
  return data

{'date': '2021-02-24T09:23:35+00:00', 'user': 'ArjunSinghPanam'}

In [ ]:
def punto1():
  S_DIR = r'/content/gdrive/My Drive/data_chan/'

  data = load_data()
  df = pd.DataFrame(data)
  #df = df[['date','user']]
  df['date'] = pd.to_datetime(df['date']).dt.date
  #df['user'] = df.apply(lambda row: row.user.get('username'), axis=1)
  dates = df.groupby(['date']).agg(count=("date", 'count')).reset_index().sort_values(by=['count'], ascending=False).head(10)['date'].to_list()
  df = df[df['date'].isin(dates)]
  df = pd.crosstab(df['date'], df['user'])
  return list(df.idxmax(axis=1).to_dict().items())

#**Punto 2**

---




In [ ]:
pd.options.display.max_colwidth = 400

In [ ]:
S_DIR = r'/content/gdrive/My Drive/data_chan/'

with open(os.path.join(S_DIR, 'farmers-protest-tweets-2021-2-4.json')) as json_file:
    data = json_file.readlines()
    data = list(map(json.loads, data))
df = pd.DataFrame(data)

In [ ]:
df[["content"]]

,content
0,The world progresses while the Indian police a...
1,#FarmersProtest \n#ModiIgnoringFarmersDeaths \...
2,ਪੈਟਰੋਲ ਦੀਆਂ ਕੀਮਤਾਂ ਨੂੰ ਮੱਦੇਨਜ਼ਰ ਰੱਖਦੇ ਹੋਏ \nਮੇ...
3,@ReallySwara @rohini_sgh watch full video here...
4,#KisanEktaMorcha #FarmersProtest #NoFarmersNoF...
...,...
117402,#FarmersProtest #KisanAndolan #KisaanMajdoorEk...
117403,PM मोदी की अपील के बीच संयुक्त किसान मोर्चा का...
117404,United we stand.\nDivided we fall\n#Mahapancha...
117405,"सिंघु बॉर्डर पर लंबी लड़ाई की तैयारी, किसानों ..."


In [ ]:
df.columns

Index(['url', 'date', 'content', 'renderedContent', 'id', 'user', 'outlinks',
       'tcooutlinks', 'replyCount', 'retweetCount', 'likeCount', 'quoteCount',
       'conversationId', 'lang', 'source', 'sourceUrl', 'sourceLabel', 'media',
       'retweetedTweet', 'quotedTweet', 'mentionedUsers'],
      dtype='object')

In [ ]:
regex = re.compile(r'\d+(.*?)[\u263a-\U0001f645]')
regex.findall(df.content)
match = text_column.str.match(pattern)

In [13]:
EMOJI_PATTERN = re.compile(
    "["
    "\U0001F1E0-\U0001F1FF"
    "\U0001F300-\U0001F5FF"
    "\U0001F600-\U0001F64F"
    "\U0001F680-\U0001F6FF"
    "\U0001F700-\U0001F77F"
    "\U0001F900-\U0001F9FF"
    "\U0001FA00-\U0001FA6F"
    "\U0001FA70-\U0001FAFF"
    "\U00002702-\U000027B0"
    "\U000024C2-\U0001F251"
    "]"
)

In [ ]:
emoji_list = df.content.str.findall(EMOJI_PATTERN).to_list()

In [ ]:
df["emojis"] = df.content.str.findall(EMOJI_PATTERN)
df[["content","emojis"]]

,content,emojis
0,The world progresses while the Indian police and Govt are still trying to take India back to the horrific past through its tyranny. \n\n@narendramodi @DelhiPolice Shame on you. \n\n#ModiDontSellFarmers \n#FarmersProtest \n#FreeNodeepKaur https://t.co/es3kn0IQAF,[]
1,"#FarmersProtest \n#ModiIgnoringFarmersDeaths \n#ModiDontSellFarmers \n@Kisanektamorcha \nFarmers constantly distroying crops throughout India. \nReally, it's hearts breaking...we care about our crops like our children. And govt. agriculture minister is laughing on us🚜🌾WE WILL WIN💪 https://t.co/kLspngG9xE","[🚜, 🌾, 💪]"
2,ਪੈਟਰੋਲ ਦੀਆਂ ਕੀਮਤਾਂ ਨੂੰ ਮੱਦੇਨਜ਼ਰ ਰੱਖਦੇ ਹੋਏ \nਮੇਰੇ ਹਿਸਾਬ ਨਾਲ ਬਾਹਰ(ਪ੍ਰਦੇਸ਼) ਜਾਣ ਨਾਲੋਂ ਬਿਹਤਰ ਆ ਭਾਰਤ 'ਚ ਪੈਟਰੋਲ ਪੰਪ ਪਾ ਲਈਏ। 🤫🤫🤔🤔\n#FarmersProtest,"[🤫, 🤫, 🤔, 🤔]"
3,@ReallySwara @rohini_sgh watch full video here https://t.co/wBPNdJdB0n\n#farmersprotest #NoFarmersNoFood https://t.co/fUsTOKOcXK,[]
4,#KisanEktaMorcha #FarmersProtest #NoFarmersNoFood https://t.co/g9TYYBHQRH,[]
...,...,...
117402,#FarmersProtest #KisanAndolan #KisaanMajdoorEktaZindabad,[]
117403,PM मोदी की अपील के बीच संयुक्त किसान मोर्चा का बहुत बड़ा ऐलान | PunjabTak\n#FarmersProtest #KisanAandolan #FarmersFury #Chakkajam #SanyukthKisanMorcha #MasalJulus #NarendraModi #DarshanPal #NarendraSinghTomar #YogendraYadav #TollFree #CandleMarch \n\nhttps://t.co/AzZNOGI8BX https://t.co/1UZvVvYK2e,[]
117404,United we stand.\nDivided we fall\n#MahapanchayatRevolution\n#FarmersProtest https://t.co/LPJkjxldM2,[]
117405,"सिंघु बॉर्डर पर लंबी लड़ाई की तैयारी, किसानों ने प्रदर्शन स्थल पर बुनियादी ढांचे को मजबूत करना किया शुरू!\n\n#FarmersProtest | #MSP | #FarmBill2020 | #SinghuBorder\nhttps://t.co/bkjh7WXc0w",[]


Se compara el texto con la lista de emojis

In [ ]:
emoji_list = list(filter(lambda x: len(x) > 0, emoji_list))
emoji_list = [emoji for row in emoji_list for emoji in row]
emoji_list = [e for e in emoji_list if e not in ['️','🏻','🏽']]


In [ ]:
len(emoji_list)

48445

In [ ]:
def calculate_tf(item_list):
  item_list = np.array(item_list)
  unique, frequency = np.unique(item_list, return_counts = True)
  return list(zip(unique,frequency))

In [ ]:

#x = np.array(emoji_list)
#unique, frequency = np.unique(x, return_counts = True)
#tf = list(zip(unique,frequency))
tf = calculate_tf(emoji_list)
tf

[('╠', 1),
 ('╣', 1),
 ('▪', 50),
 ('▶', 5),
 ('►', 9),
 ('◆', 2),
 ('○', 1),
 ('●', 12),
 ('◼', 2),
 ('◾', 1),
 ('☀', 12),
 ('★', 5),
 ('☑', 9),
 ('☕', 15),
 ('☘', 7),
 ('☛', 2),
 ('☝', 32),
 ('☠', 48),
 ('☪', 9),
 ('☬', 19),
 ('☭', 2),
 ('☮', 316),
 ('☯', 1),
 ('☸', 1),
 ('☹', 35),
 ('☺', 26),
 ('☻', 1),
 ('♀', 148),
 ('♂', 184),
 ('♉', 1),
 ('♒', 1),
 ('♟', 2),
 ('♠', 8),
 ('♡', 4),
 ('♥', 81),
 ('♦', 3),
 ('♨', 2),
 ('♻', 22),
 ('♾', 2),
 ('⚔', 55),
 ('⚕', 1),
 ('⚖', 4),
 ('⚘', 31),
 ('⚠', 29),
 ('⚡', 9),
 ('⚰', 3),
 ('⛈', 1),
 ('⛏', 4),
 ('⛔', 6),
 ('⛩', 1),
 ('⛳', 68),
 ('⛺', 3),
 ('⛽', 35),
 ('✅', 57),
 ('✈', 3),
 ('✉', 2),
 ('✊', 2411),
 ('✋', 9),
 ('✌', 274),
 ('✍', 103),
 ('✒', 2),
 ('✓', 4),
 ('✔', 30),
 ('✖', 2),
 ('✞', 1),
 ('✨', 40),
 ('✽', 8),
 ('✿', 2),
 ('❌', 32),
 ('❎', 4),
 ('❓', 54),
 ('❕', 3),
 ('❗', 32),
 ('❝', 1),
 ('❞', 1),
 ('❣', 52),
 ('❤', 1779),
 ('➕', 1),
 ('➡', 56),
 ('➰', 2),
 ('⠀', 3),
 ('⤵', 23),
 ('⬅', 2),
 ('⬆', 8),
 ('⬇', 37),
 ('⬛', 1),
 ('⭐', 6),
 

In [ ]:
tf.sort(key=lambda t: t[1], reverse = True)
tf[:10]

[('🙏', 7286),
 ('😂', 3072),
 ('🚜', 2972),
 ('✊', 2411),
 ('🌾', 2363),
 ('🇮', 2096),
 ('🇳', 2094),
 ('❤', 1779),
 ('🤣', 1668),
 ('👇', 1108)]

### Memory

In [32]:
file_path_ = '/content/gdrive/My Drive/data_chan/farmers-protest-tweets-2021-2-4.json'
def load_data(file_path):
  with open(file_path) as json_file:
      data = json_file.readlines()
      data = list(map(json.loads, data))
  return data

def calculate_tf(item_list):
  item_list = np.array(item_list)
  term, frequency = np.unique(item_list, return_counts = True)
  return list(zip(term,frequency))

def punto2_memory(file_path):
  EMOJI_PATTERN = re.compile(
      "["
      "\U0001F1E0-\U0001F1FF"
      "\U0001F300-\U0001F5FF"
      "\U0001F600-\U0001F64F"
      "\U0001F680-\U0001F6FF"
      "\U0001F700-\U0001F77F"
      "\U0001F900-\U0001F9FF"
      "\U0001FA00-\U0001FA6F"
      "\U0001FA70-\U0001FAFF"
      "\U00002702-\U000027B0"
      "\U000024C2-\U0001F251"
      "]"
  )
  data = load_data(file_path)
  df = pd.DataFrame(data)
  df = df[['content']]
  df['content'] = df['content'].astype("string")
  emoji_list = df.content.str.findall(EMOJI_PATTERN).to_list()
  emoji_list = list(filter(lambda x: len(x) > 0, emoji_list))
  emoji_list = [emoji for row in emoji_list for emoji in row]
  emoji_list = [e for e in emoji_list if e not in ['️','🏻','🏽']]
  tf = calculate_tf(emoji_list)
  tf.sort(key=lambda t: t[1], reverse = True)
  return tf[:10]

In [33]:
punto2_memory('/content/gdrive/My Drive/data_chan/farmers-protest-tweets-2021-2-4.json')

<ipython-input-32-7f080e97f512>:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['content'] = df['content'].astype("string")


[('🙏', 7286),
 ('😂', 3072),
 ('🚜', 2972),
 ('✊', 2411),
 ('🌾', 2363),
 ('🇮', 2096),
 ('🇳', 2094),
 ('❤', 1779),
 ('🤣', 1668),
 ('👇', 1108)]

### Time

In [34]:
file_path_ = '/content/gdrive/My Drive/data_chan/farmers-protest-tweets-2021-2-4.json'
def load_data(file_path):
  with open(file_path) as json_file:
      data = json_file.readlines()
      data = list(map(json.loads, data))
  return data

def calculate_tf(item_list):
  item_list = np.array(item_list)
  term, frequency = np.unique(item_list, return_counts = True)
  return list(zip(term,frequency))

def punto2_time(file_path):
  EMOJI_PATTERN = re.compile(
      "["
      "\U0001F1E0-\U0001F1FF"
      "\U0001F300-\U0001F5FF"
      "\U0001F600-\U0001F64F"
      "\U0001F680-\U0001F6FF"
      "\U0001F700-\U0001F77F"
      "\U0001F900-\U0001F9FF"
      "\U0001FA00-\U0001FA6F"
      "\U0001FA70-\U0001FAFF"
      "\U00002702-\U000027B0"
      "\U000024C2-\U0001F251"
      "]"
  )
  data = load_data(file_path)
  df = pd.DataFrame(data)
  emoji_list = df.content.str.findall(EMOJI_PATTERN).to_list()
  emoji_list = list(filter(lambda x: len(x) > 0, emoji_list))
  emoji_list = [emoji for row in emoji_list for emoji in row]
  emoji_list = [e for e in emoji_list if e not in ['️','🏻','🏽']]
  tf = calculate_tf(emoji_list)
  tf.sort(key=lambda t: t[1], reverse = True)
  return tf[:10]

## Prueba estrategia de paralelizacion para cargar los datos utilizando Multiprocessing

In [30]:

def load_data_mp(file_path):
  with open(file_path) as json_file:
      data = json_file.readlines()
      with Pool(cpu_count()) as p:
        data = list(p.map(json.loads, data))
  return data

def load_data(file_path):
  with open(file_path) as json_file:
      data = json_file.readlines()
      data = list(map(json.loads, data))
  return data

In [29]:
start_time = time.time()
load_data_mp(file_path)[0]
print("--- %s ---" % (time.time() -start_time))

--- 31.889872550964355 ---


In [31]:
start_time = time.time()
load_data(file_path)[0]
print("--- %s ---" % (time.time() -start_time))

--- 13.517795324325562 ---


Se descarta estrategia de paralelizacion utilizando libreria multiprocessing

#**Punto 3**

In [ ]:
S_DIR = r'/content/gdrive/My Drive/data_chan/'

with open(os.path.join(S_DIR, 'farmers-protest-tweets-2021-2-4.json')) as json_file:
    data = json_file.readlines()
    data = list(map(lambda x: json.loads(x).get("mentionedUsers"), data))
data[0]

[{'username': 'narendramodi',
  'displayname': 'Narendra Modi',
  'id': 18839785,
  'description': None,
  'rawDescription': None,
  'descriptionUrls': None,
  'verified': None,
  'created': None,
  'followersCount': None,
  'friendsCount': None,
  'statusesCount': None,
  'favouritesCount': None,
  'listedCount': None,
  'mediaCount': None,
  'location': None,
  'protected': None,
  'linkUrl': None,
  'linkTcourl': None,
  'profileImageUrl': None,
  'profileBannerUrl': None,
  'url': 'https://twitter.com/narendramodi'},
 {'username': 'DelhiPolice',
  'displayname': '#DilKiPolice Delhi Police',
  'id': 1850705408,
  'description': None,
  'rawDescription': None,
  'descriptionUrls': None,
  'verified': None,
  'created': None,
  'followersCount': None,
  'friendsCount': None,
  'statusesCount': None,
  'favouritesCount': None,
  'listedCount': None,
  'mediaCount': None,
  'location': None,
  'protected': None,
  'linkUrl': None,
  'linkTcourl': None,
  'profileImageUrl': None,
  'prof

In [ ]:
referenced_users = list(filter(lambda x: x is not None, data))

In [ ]:
referenced_users = [user.get("username") for row in referenced_users for user in row]


In [ ]:
tf = calculate_tf(referenced_users)

In [ ]:
tf.sort(key=lambda t: t[1], reverse = True)
tf[:10]

[('narendramodi', 2265),
 ('Kisanektamorcha', 1840),
 ('RakeshTikaitBKU', 1644),
 ('PMOIndia', 1427),
 ('RahulGandhi', 1146),
 ('GretaThunberg', 1048),
 ('RaviSinghKA', 1019),
 ('rihanna', 986),
 ('UNHumanRights', 962),
 ('meenaharris', 926)]

In [44]:
def load_data(file_path):
  with open(file_path) as json_file:
      data = json_file.readlines()
      data = list(map(lambda x: json.loads(x).get("mentionedUsers"), data))
  return data

def punto3_memory(file_path):
  data = load_data(file_path)
  referenced_users = list(filter(lambda x: x is not None, data))
  referenced_users = [user.get("username") for row in referenced_users for user in row]
  tf = calculate_tf(referenced_users)
  tf.sort(key=lambda t: t[1], reverse = True)
  return tf[:10]

In [45]:
punto3_memory(file_path)

[('narendramodi', 2265),
 ('Kisanektamorcha', 1840),
 ('RakeshTikaitBKU', 1644),
 ('PMOIndia', 1427),
 ('RahulGandhi', 1146),
 ('GretaThunberg', 1048),
 ('RaviSinghKA', 1019),
 ('rihanna', 986),
 ('UNHumanRights', 962),
 ('meenaharris', 926)]